## <b><font color='darkblue'>Preface</font></b>
([article source](https://github.com/karndeepsingh/ApplicationsBuildWithLLMs/blob/main/Langchain_With_Gemini_And_Build_RAG.ipynb), [YT](https://www.youtube.com/watch?v=8xVmzoP1lks)) <b><font size='3ptx'>Here we are going to demonstrate the usage of LLM Gemini + RAG.</font></b>

### <b><font color='darkgreen'>Setup environment</font></b>

In [1]:
! pip install -q --upgrade google-generativeai langchain-google-genai chromadb pypdf


[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [39]:
import os
import google.generativeai as genai
from IPython.display import display
from IPython.display import Markdown
import textwrap
from dotenv import load_dotenv

# loads the .env file
load_dotenv()

GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")
genai.configure(api_key=GOOGLE_API_KEY)

In [7]:
def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

## <b><font color='darkblue'>Text Generation</font></b>

In [9]:
model = genai.GenerativeModel(model_name = "gemini-pro")
model

genai.GenerativeModel(
    model_name='models/gemini-pro',
    generation_config={},
    safety_settings={},
    tools=None,
    system_instruction=None,
)

In [10]:
response = model.generate_content("What are the usecases of LLMs?")

In [11]:
to_markdown(response.text)

> **Content Creation:**
> 
> * **Text generation:** Creating original stories, articles, marketing copy, and social media posts.
> * **Code generation:** Assisting programmers in writing code, debugging, and generating documentation.
> * **Translation:** Translating text between different languages.
> * **Summarization:** Condensing long pieces of text into concise summaries.
> * **Chatbot and conversational AI:** Powering chatbots and virtual assistants to engage in natural language conversations.
> 
> **Data Analysis and Research:**
> 
> * **Text and sentiment analysis:** Identifying emotions, opinions, and themes in text.
> * **Data exploration and insights:** Uncovering patterns, insights, and trends from large datasets.
> * **Question answering:** Answering questions based on a large repository of knowledge.
> * **Fact-checking and bias detection:** Identifying bias and verifying factual claims.
> 
> **Business Operations:**
> 
> * **Customer service:** Assisting customer service representatives with resolving queries and providing support.
> * **Sales and marketing:** Generating targeted marketing copy, analyzing customer feedback, and predicting customer behavior.
> * **Process automation:** Automating repetitive tasks, such as data entry, email writing, and form processing.
> * **Collaboration and knowledge management:** Facilitating collaboration, sharing information, and managing knowledge across teams.
> 
> **Education and Training:**
> 
> * **Personalized learning:** Adapting learning content to individual student's needs and progress.
> * **Language learning:** Assisting language learners with grammar, vocabulary acquisition, and pronunciation.
> * **Code literacy:** Teaching coding concepts and helping students build projects.
> * **Critical thinking and problem-solving:** Engaging students in thought-provoking discussions and problem-solving exercises.
> 
> **Other Usecases:**
> 
> * **Gaming and entertainment:** Generating unique and engaging gameplay experiences, such as procedural levels and interactive stories.
> * **Medical research:** Assisting in drug discovery, disease diagnosis, and personalized treatment plans.
> * **Financial modeling and forecasting:** Predicting financial trends, analyzing market data, and making investment recommendations.
> * **Transportation and logistics:** Optimizing routes, forecasting traffic patterns, and managing inventory.
> * **Environmental science:** Modeling climate change, predicting natural disasters, and analyzing data from sensors.

## <b><font color='darkblue'>Use LangChain to Access Gemini API</font></b>

In [12]:
from langchain_google_genai import ChatGoogleGenerativeAI

In [13]:
llm = ChatGoogleGenerativeAI(
    model="gemini-pro", google_api_key=GOOGLE_API_KEY)

In [16]:
%%time
result = llm.invoke("What are the usecases of LLMs?")

CPU times: user 16.6 ms, sys: 1.13 ms, total: 17.8 ms
Wall time: 6.57 s


In [15]:
to_markdown(result.content)

> **Language Generation and Summarization:**
> 
> * Automated content creation, such as news articles, blog posts, and marketing copy
> * Text summarization and abstraction
> * Chatbot and virtual assistant responses
> 
> **Code Generation and Debugging:**
> 
> * Generating code snippets and functions in various programming languages
> * Debugging and fixing errors in code
> * Automating software testing
> 
> **Translation and Localization:**
> 
> * Translating text between different languages
> * Localizing content for specific regions and cultures
> 
> **Search and Information Retrieval:**
> 
> * Improving search engine results by understanding natural language queries
> * Extracting key information from large text corpora
> * Question answering and fact verification
> 
> **Content Analysis and Classification:**
> 
> * Sentiment analysis and emotion detection
> * Topic modeling and text classification
> * Detecting fake news and misinformation
> 
> **Education and Learning:**
> 
> * Personalized learning assistants
> * Language learning apps
> * Educational content generation
> 
> **Healthcare and Medicine:**
> 
> * Medical diagnosis and treatment recommendations
> * Drug discovery and research
> * Analyzing medical records and patient data
> 
> **Finance and Business:**
> 
> * Financial analysis and forecasting
> * Market research and sentiment analysis
> * Contract and legal document review
> 
> **Customer Service and Support:**
> 
> * Chatbots and virtual assistants for customer inquiries
> * Complaint and feedback analysis
> * Knowledge base creation and maintenance
> 
> **Entertainment and Creative Arts:**
> 
> * Generating story ideas and plot outlines
> * Creating music lyrics and melodies
> * Developing video game scripts and dialogue
> 
> **Other Use Cases:**
> 
> * Social media monitoring and analysis
> * Spam and phishing detection
> * Data extraction and cleaning
> * Language modeling for natural language processing tasks

## <b><font color='darkblue'>Chat with Documents using RAG (Retreival Augment Generation)</font></b>

In [70]:
import urllib
import warnings
from pathlib import Path as p
from pprint import pprint

import pandas as pd
from langchain import PromptTemplate
from langchain.chains.question_answering import load_qa_chain
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.embeddings import SentenceTransformerEmbeddings

warnings.filterwarnings("ignore")
# restart python kernal if issues with langchain import.

In [20]:
model = ChatGoogleGenerativeAI(
    model="gemini-pro",google_api_key=GOOGLE_API_KEY,
    temperature=0.2,convert_system_message_to_human=True)

### <b><font color='darkgreen'>RAG Pipeline: Embedding + Gemini (LLM)</font></b>

In [40]:
import chromadb
from langchain.vectorstores import Chroma
from chromadb.utils import embedding_functions

CHROMA_PATH = os.getenv('CHROMA_PATH')
COLLECTION_NAME = os.getenv('COLLECTION_NAME')
EMBEDDING_FUNC_NAME = os.getenv('EMBEDDING_FUNC_NAME')

In [41]:
# print(CHROMA_PATH)

In [29]:
def get_chroma_collection( 
    chroma_path: str=CHROMA_PATH, collection_name: str=COLLECTION_NAME):
  chroma_client = chromadb.PersistentClient(chroma_path)
  for collection in chroma_client.list_collections():
    if collection.name == collection_name:
      print(f'Found collection with name="{collection_name}"!')
      return collection

  raise Exception(
      'Chroma collection with name="{collection_name}" does not exist!')

In [71]:
client = chromadb.PersistentClient(CHROMA_PATH)
#embedding_func = embedding_functions.SentenceTransformerEmbeddingFunction(
#      model_name=EMBEDDING_FUNC_NAME)
embeddings = SentenceTransformerEmbeddings(model_name=EMBEDDING_FUNC_NAME)

langchain_chroma =  Chroma(
    client=client,
    collection_name=COLLECTION_NAME,
    embedding_function=embeddings)

In [52]:
vector_index = langchain_chroma.as_retriever(search_kwargs={"k":5})

In [53]:
qa_chain = RetrievalQA.from_chain_type(
    model,
    retriever=vector_index,
    return_source_documents=True)

In [72]:
%%time
question = "Could you give me the sample code in using BDS broker from BTTC?"
result = qa_chain({"query": question})

CPU times: user 56.2 ms, sys: 2.8 ms, total: 59 ms
Wall time: 4.87 s


In [73]:
result["result"]

'We call method `pair` on BDS broker to pair BDS device with DUT device:\n```python\n# Triggering pairing between BDS device and DUT device:\n>>> bds_broker.pair()\nTrue\n\n# The returned `True` indicated that BDS device and DUT device are paired:\n>>> bds_broker.is_bt_paired()\nTrue\n```\nthanks for asking!'

In [64]:
# Show source document(s)
# Markdown(result["source_documents"][0].page_content)

In [74]:
Markdown(result["result"])

We call method `pair` on BDS broker to pair BDS device with DUT device:
```python
# Triggering pairing between BDS device and DUT device:
>>> bds_broker.pair()
True

# The returned `True` indicated that BDS device and DUT device are paired:
>>> bds_broker.is_bt_paired()
True
```
thanks for asking!

In [75]:
template = """You are a software engineer with excellent experience in using bttc.
If you don't know the answer, just say that you don't know, don't try to make up an answer.
Keep the answer as concise as possible. Always say "thanks for asking!" at the end of the answer.
Use the following collected information to answer questions:
{context}

Question: {question}
Helpful Answer:"""
QA_CHAIN_PROMPT = PromptTemplate.from_template(template)# Run chain
qa_chain = RetrievalQA.from_chain_type(
    model,
    retriever=vector_index,
    return_source_documents=True,
    chain_type_kwargs={"prompt": QA_CHAIN_PROMPT}
)

In [76]:
%%time
question = "Could you give me the sample code in using Dialer simulator from BTTC?"
result = qa_chain({"query": question})

CPU times: user 44.7 ms, sys: 41.9 ms, total: 86.5 ms
Wall time: 12.4 s


In [77]:
Markdown(result["result"])

```python
# Gets the DUT device from its serial.
>>> import bttc
>>> dut = bttc.get('36121FDJG000GR')

# Import the module `hfp_devices`
>>> from bttc.profiles.hfp import hfp_devices

# Create an instance of AndroidPhoneWithDialerSimulator, wrapping the DUT
>>> dut_with_dialer_simulator = hfp_devices.AndroidPhoneWithDialerSimulator(dut)
>>> dut_with_dialer_simulator.ds
<bttc.utils.dialer_simulator.DialerSimulator object at 0x7fdf5bdc7fd0>

# Check the initial call state (should be `IDLE`)
>>> dut_with_dialer_simulator.get_call_state()
<CallStateEnum.IDLE: 'IDLE'>

# Simulate an incoming call
>>> dut_with_dialer_simulator.incoming_call()
CallResult(caller=..., callee=<bttc.profiles.hfp.hfp_devices.AndroidPhoneWithDialerSimulator object at 0x7f79aabe4550>, error=None)

# Verify the call state changed to `RINGING`
>>> dut_with_dialer_simulator.get_call_state()
<CallStateEnum.RINGING: 'RINGING'>

# Simulate answering the call
>>> dut_with_dialer_simulator.answer_call()
True

# Verify the call state changed to `ACTIVE`
>>> dut_with_dialer_simulator.get_call_state()
<CallStateEnum.ACTIVE: 'ACTIVE'>

# Ends the call and expects call state to be `IDLE`.
>>> dut_with_dialer_simulator.end_call()
True
>>> dut_with_dialer_simulator.get_call_state()
<CallStateEnum.IDLE: 'IDLE'>

# Simulates outgoing call and expects call state to be `ACTIVE`.
>>> dut_with_dialer_simulator.outgoing_call()
CallResult(caller=<bttc.profiles.hfp.hfp_devices.AndroidPhoneWithDialerSimulator object at 0x7f79aabe4550>, callee=..., error=None)
>>> dut_with_dialer_simulator.get_call_state()
<CallStateEnum.ACTIVE: 'ACTIVE'>

# Ends the call and expects call state to be `IDLE`
>>> dut_with_dialer_simulator.end_call()
True
>>> dut_with_dialer_simulator.get_call_state()
<CallStateEnum.IDLE: 'IDLE'>
```
Thanks for asking!

In [78]:
question = "Describe Random forest?"
result = qa_chain({"query": question})

In [79]:
Markdown(result["result"])

I'm sorry, but the provided context does not mention anything about Random forest, so I cannot answer this question.
Thanks for asking!

## <b><font color='darkblue'>Supplement</font></b>
* [Stackoverflow - How to connect my ChromaDB collection and using langchain](https://stackoverflow.com/questions/77428241/how-to-connect-my-chromadb-collection-and-using-langchain)
* [Stackoverflow - Chromadb + Langchain + SentenceTransformerEmbeddingFunction throwing 'SentenceTransformerEmbeddingFunction' object has no attribute 'embed_documents'](https://stackoverflow.com/questions/77004874/chromadb-langchain-sentencetransformerembeddingfunction-throwing-sentencetr)